## Abstract Attributes

Not working yet.

In [120]:
def abstractattrs(*attrs):
    def wrapper(cls):
        
        def method(cls, *args, **kwargs):
            super(cls, cls).__init_subclass__(*args, **kwargs)
            for attr in attrs:
                if not hasattr(cls, attr):
                    raise RuntimeError(f'Class must have attribute {attr}.') 
                    
        setattr(cls, '__init_subclass__', classmethod(method))
        return cls
    return wrapper              

In [121]:
@abstractattrs('name', 'age')
class Base:
    def __init__(self, age=7):
        self.age = age 

In [122]:
class Child(Base):
    a = 'a'

RecursionError: maximum recursion depth exceeded while calling a Python object

In [118]:
class AbstractAttrs:
    
    def __init__(self, *args):
        print(args)
        self.args = list(args)
    
    @classmethod
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
#         if not hasattr(cls, f'_{cls.__name__}__attrs'):
#             print('no attrs')
        print('init subclass', AbstractAttrs._args())
        for arg in AbstractAttrs.args:
            assert hasattr(cls, arg), \
                f'{cls.__name__} must have class variable {arg}.'

In [119]:
class Callback(AbstractAttrs):
    pass

TypeError: _args() missing 1 required positional argument: 'self'

In [112]:
c = Callback('order', 'is_val')

('order', 'is_val')


In [113]:
class NewCallback(Callback):
    
    def __init__(self, a, order=3):
        self.a = a
        self.order = order

pre super
NewCallback
no attrs


In [107]:
nc = NewCallback('a', 'my_order')

In [108]:
nc

In [109]:
nc._attrs

['order', 'val_loop']

In [15]:
from abc import ABC, abstractmethod, abstractproperty

In [16]:
class Abstract(ABC):
    def __init__(self, a):
        self.a = a
    @abstractmethod
    def walk(self):
        pass
    
    @abstractproperty
    def b(self):
        pass

In [51]:
class Child(Abstract):
    b = 3
        
    def walk(self):
        return self.a
    
#     def b(self):
#         print('in b, method not prop')
#         return self.a*2

In [52]:
c = Child(3)

In [54]:
c.b

3

In [91]:
from inspect import ismethod

In [429]:
class AbstractAttrs(type):
    """Basically the attribute equivalent of abc.abstractmethod: this allows 
    us to define an abstract parent class that requires its children to 
    possess certain class and/or instance attributes. This differs from
    abc.abstractproperty in a few ways:
    
    1. abstractproperty ignores instance attributes. AbstractAttrs lets us
    specify required instance attributes and/or class attributes.
    3. abstractproperty considers the requirement fulfilled by methods,
    properties, and class attributes. AbstractAttrs does not allow methods
    to fulfill instance attribute requirements, thought properties will 
    fulfill requirements for either instance or class attributes.
    """

    def __new__(cls, name, bases, methods, **meta_kwargs):
        print('meth', methods)
        class_ = type.__new__(cls, name, bases, methods)
        class_attrs = meta_kwargs.get('class_attrs', [])
        inst_attrs = meta_kwargs.get('inst_attrs', [])
        
        def __init_subclass__(cls, **kwargs):
            super().__init_subclass__(**kwargs)
            for attr in class_attrs:
                # TypeError maintains consistency with abstractmethod.
                if not hasattr(cls, attr):
                    raise TypeError(f'{cls} must have class attribute '
                                    f'`{attr}`.')
        
        # Make sure we distinguish between the abstract parent class that 
        # defines an interface and the child classes that implement it. The
        # abstract parent should not define the required attributes: it merely
        # enforces the requirement that its children do. We want the children
        # to inherit class_attrs and inst_attrs without overwriting them when
        # their own __new__ is called, so that AbstractAttrs.__call__ can
        # use them for validation. Only change this if you're very confident
        # you understand the repercussions.
        if class_attrs or inst_attrs:
            class_.__init_subclass__ = classmethod(__init_subclass__)
            class_._is_parent = True
            class_.class_attrs = class_attrs
            class_.inst_attrs = inst_attrs
        else:
            class_._is_parent = False
        return class_

    def __call__(cls, *args, **kwargs):
        inst = cls.__new__(cls, *args, **kwargs)
        if not isinstance(inst, cls): return inst
        
        inst.__init__(*args, **kwargs)
        if cls._is_parent: return inst
        
        # Validate children.
        for attr in inst.inst_attrs:
            # TypeError maintains consistency with abstractmethod.
            if not hasattr(inst, attr):
                raise TypeError(f'Instances of {type(inst)} must '
                                f'have instance attribute `{attr}`.')
            elif ismethod(getattr(inst, attr)):
                raise TypeError(f'`{attr}` must be an instance attribute, '
                                'not a method.')
                
        # In AbstractAttrs.__new__, methods are still unbound so we couldn't
        # easily check this until now.
        for attr in inst.class_attrs:
            # Retrieve from inst here, not cls. cls.method is a function,
            # inst.method is a method.
            # TODO: check what happens with classmethod/staticmethod.
            if ismethod(getattr(inst, attr)):
                raise TypeError(f'`{attr}` must be a class attribute, not a '
                                'method.')
        return inst

In [414]:
class Parent(metaclass=AbstractAttrs, inst_attrs=['name'],
             class_attrs=['order', 'is_val']):
    pass

meth {'__module__': '__main__', '__qualname__': 'Parent'}


In [415]:
p = Parent()

In [416]:
p.class_attrs, p.inst_attrs, p._is_parent

(['order', 'is_val'], ['name'], True)

In [417]:
# Implements required class vars but not required inst vars. Error will be
# thrown on object instantiation, not class definition.
class Child(Parent):
    
    order = 33
    is_val = True
    
    def __init__(self, a):
        self.a = a
        print('child init')

meth {'__module__': '__main__', '__qualname__': 'Child', 'order': 33, 'is_val': True, '__init__': <function Child.__init__ at 0x10a4fb840>}


In [418]:
c = Child(4)
c

child init


TypeError: Instances of <class '__main__.Child'> must have instance attribute `name`.

In [419]:
# Possesses one of the required class vars but the other is defined as an
# instance var. Throws error at class definition. If we want to accept either,
# we could use abc.abstractmethod or simply define our instance variables.
class BadChild(Parent):
    
    order = 1
    
    def __init__(self, b, is_val):
        self.b = b
        self.is_val = is_val

meth {'__module__': '__main__', '__qualname__': 'BadChild', 'order': 1, '__init__': <function BadChild.__init__ at 0x10abb2840>}


TypeError: <class '__main__.BadChild'> must have class attribute `is_val`.

In [420]:
# Trying to sneakily use a method to fulfill an instance var requirement.
class TrickyChild(Parent):
    
    order = 33
    is_val = True
    
    def __init__(self, a):
        self.a = a
        print('child init')
        
    def name(self):
        print('in name')

meth {'__module__': '__main__', '__qualname__': 'TrickyChild', 'order': 33, 'is_val': True, '__init__': <function TrickyChild.__init__ at 0x10a73f840>, 'name': <function TrickyChild.name at 0x10a73fea0>}


In [421]:
tc = TrickyChild(44)

child init


TypeError: `name` must be an instance attribute, not a method.

In [422]:
# Trying to use a property to fulfill an instance var requirement 
# (acceptable).
class TrickyChild2(Parent):
    
    order = 33
    is_val = True
    
    def __init__(self, a):
        self.a = a
        print('child init')
        
    @property
    def name(self):
        print('in name')

meth {'__module__': '__main__', '__qualname__': 'TrickyChild2', 'order': 33, 'is_val': True, '__init__': <function TrickyChild2.__init__ at 0x10a12c7b8>, 'name': <property object at 0x10a1bb688>}


In [423]:
tc = TrickyChild2(44)
tc.name

child init
in name
in name
in name


In [424]:
# Trying to use a property to fulfill a class var requirement. Acceptable.
class TrickyChild3(Parent):
    
    order = 33
    
    def __init__(self, a, name):
        self.a = a
        self.name = name
        print('child init')
        
    @property
    def is_val(self):
        return 'is val'

meth {'__module__': '__main__', '__qualname__': 'TrickyChild3', 'order': 33, '__init__': <function TrickyChild3.__init__ at 0x10abb2bf8>, 'is_val': <property object at 0x10a3032c8>}


In [425]:
tc = TrickyChild3(44, 'sam')

child init


In [430]:
# Trying to use a method to fulfill a class var req. UNacceptable.
class TrickyChild4(Parent):
    
    order = 33
    
    def __init__(self, a, name):
        self.a = a
        self.name = name
        print('child init')
        
    def is_val(self):
        print(self, type(self))
        return self.a, 'is_val'
        
    def testing(self):
        return self.a*2, 'is_val'

meth {'__module__': '__main__', '__qualname__': 'TrickyChild4', 'order': 33, '__init__': <function TrickyChild4.__init__ at 0x10a1c5950>, 'is_val': <function TrickyChild4.is_val at 0x10a1c5730>, 'testing': <function TrickyChild4.testing at 0x10aee5b70>}


In [431]:
tc = TrickyChild4(44, 'sam')

child init


TypeError: `is_val` must be class attribute, not amethod.

In [432]:
# Implements desired interface.
class GoodChild(Parent):
    
    order = 100
    is_val = False
    
    def __init__(self, name):
        self.name = name
        
    def walk(self):
        return self.order, self.is_val, self.name

meth {'__module__': '__main__', '__qualname__': 'GoodChild', 'order': 100, 'is_val': False, '__init__': <function GoodChild.__init__ at 0x10aee5840>, 'walk': <function GoodChild.walk at 0x10aee5d08>}


In [433]:
ian = GoodChild('Ian')

In [434]:
ian.walk()

(100, False, 'Ian')

## LSH

In [30]:
from datasketch import lsh, MinHashLSH, MinHash, MinHashLSHForest

from htools import *

In [2]:
set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)

In [3]:
def ngrams(word, n=3, step=1, drop_last=False):
    """To get non-overlapping sequences, pass in same value for `step` as `n`."""
    stop = max(1, step+len(word)-n)
    ngrams_ = []
    for i in range(0, stop, step):
        ngrams_.append(word[i:i+n])
    if drop_last and len(ngrams_[-1]) < n: ngrams_ = ngrams_[:-1]
    return ngrams_

In [35]:
w = 'dinosaur'
tw = 'I'
ws = ['dino', 'rhino', 'dog', 'cat', 'lion', 'tyrannosaurus rex',
      'tiger', 'owl', 'rhinocerous', 'kino']
s = 'The mailman was running late today, possibly due to the heavy '\
    'snowfall from the previous night.'

In [6]:
ngrams(w)

['din', 'ino', 'nos', 'osa', 'sau', 'aur']

In [7]:
ngrams(w, 2)

['di', 'in', 'no', 'os', 'sa', 'au', 'ur']

In [9]:
ngrams(w, 3, 3)

['din', 'osa', 'ur']

In [12]:
ngrams(w, 3, 3, True)

['din', 'osa']

In [37]:
ngrams(s)

['The',
 'he ',
 'e m',
 ' ma',
 'mai',
 'ail',
 'ilm',
 'lma',
 'man',
 'an ',
 'n w',
 ' wa',
 'was',
 'as ',
 's r',
 ' ru',
 'run',
 'unn',
 'nni',
 'nin',
 'ing',
 'ng ',
 'g l',
 ' la',
 'lat',
 'ate',
 'te ',
 'e t',
 ' to',
 'tod',
 'oda',
 'day',
 'ay,',
 'y, ',
 ', p',
 ' po',
 'pos',
 'oss',
 'ssi',
 'sib',
 'ibl',
 'bly',
 'ly ',
 'y d',
 ' du',
 'due',
 'ue ',
 'e t',
 ' to',
 'to ',
 'o t',
 ' th',
 'the',
 'he ',
 'e h',
 ' he',
 'hea',
 'eav',
 'avy',
 'vy ',
 'y s',
 ' sn',
 'sno',
 'now',
 'owf',
 'wfa',
 'fal',
 'all',
 'll ',
 'l f',
 ' fr',
 'fro',
 'rom',
 'om ',
 'm t',
 ' th',
 'the',
 'he ',
 'e p',
 ' pr',
 'pre',
 'rev',
 'evi',
 'vio',
 'iou',
 'ous',
 'us ',
 's n',
 ' ni',
 'nig',
 'igh',
 'ght',
 'ht.']

In [13]:
def lsh_hash_word(word, num_perm=128, **ngram_kwargs):
    mhash = MinHash(num_perm=num_perm)
    for ng in ngrams(word, **ngram_kwargs):
        mhash.update(ng.encode('utf8'))
    return mhash

In [16]:
m = lsh_hash_word(w)

In [22]:
forest = MinHashLSHForest(num_perm=128)

for i, word in enumerate(ws):
    forest.add(i, lsh_hash_word(word))
forest.index()

In [24]:
forest.query(lsh_hash_word(w), 3)

[8, 0, 5]

In [26]:
ws

['dino',
 'rhino',
 'dog',
 'cat',
 'lion',
 'tyrannosaurus rex',
 'tiger',
 'owl',
 'rhinocerous',
 'kino']

In [28]:
idx = forest.query(lsh_hash_word(w), 3)
idx

[8, 0, 5]

In [32]:
select(ws, idx)

['dino', 'tyrannosaurus rex', 'rhinocerous']